In [1]:
from models.encdec_model import EncDecModel
from models.stacked_rnn import Stacked_RNN
from src.dataset_module import SolarEfficientDataset
from src.preprocessing import SolarPreprocessor
from utils.experiment_manager import ExperimentManager

import torch
from torchinfo import summary
import pandas as pd
from utils.early_stopping import EarlyStopping

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from codecarbon import EmissionsTracker

In [4]:
def clean_and_save(input_path, output_path):
    print(f"🧹 Iniciando limpeza do arquivo: {input_path}")
    
    # 1. Carrega os dados
    temp = pd.read_csv(input_path)
    df = temp[['Date_Time','Pot_BT','Irradiação Global horária(horizontal) kWh/m2',
               'Irradiação Difusa horária kWh/m2','Irradiação Global horária(Inclinada 27°) kWh/m2',
               'Temperatura ambiente °C','Umidade Relativa %','Velocidade média do vento m/s',
               'Pressão Baromêtrica mm Hg','Pluviômetro mm']].copy()
    
    print(f"   Linhas originais: {len(df)}")
    
    # 2. Garante que Date_Time é data real
    # Ajuste o nome da coluna se for diferente, mas pelo que vi é 'Date_Time'
    if 'Date_Time' in df.columns:
        df['Date_Time'] = pd.to_datetime(df['Date_Time'])
    else:
        print("❌ Erro: Coluna 'Date_Time' não encontrada.")
        return

    # 3. A GRANDE LIMPEZA: Remove duplicatas baseadas na DATA
    # keep='first' -> Mantém a primeira aparição e apaga as repetidas
    df_clean = df.drop_duplicates(subset=['Date_Time'], keep='first')
    
    # 4. Ordena cronologicamente (CRUCIAL para séries temporais)
    df_clean = df_clean.sort_values(by='Date_Time')
    
    # 5. Define como índice só para garantir que não tem erro, depois reseta
    df_clean = df_clean.set_index('Date_Time')
    
    # Verifica duplicatas no índice uma última vez
    if df_clean.index.duplicated().any():
        print(f"   ⚠️ Ainda encontrei duplicatas no índice. Removendo à força...")
        df_clean = df_clean[~df_clean.index.duplicated(keep='first')]
    
    # Reseta o index para salvar o CSV com a coluna Date_Time explícita
    df_clean = df_clean.reset_index()

    # 6. Salva o arquivo novo
    df_clean.to_csv(output_path, index=False)
    
    removidas = len(df) - len(df_clean)
    print(f"   ✅ Arquivo limpo salvo em: {output_path}")
    print(f"   🗑️  Linhas duplicadas removidas: {removidas}")
    print(f"   📊 Linhas finais: {len(df_clean)}")

In [5]:
if __name__ == "__main__":
    # --- CONFIGURAÇÃO ---
    # Coloque aqui o caminho do seu arquivo problemático
    ARQUIVO_SUJO = "data/pv0.csv"  
    
    # Nome do novo arquivo que vamos usar daqui pra frente
    ARQUIVO_LIMPO = "data/pv0_.csv" 
    
    clean_and_save(ARQUIVO_SUJO, ARQUIVO_LIMPO)

🧹 Iniciando limpeza do arquivo: data/pv0.csv
   Linhas originais: 66243
   ✅ Arquivo limpo salvo em: data/pv0_.csv
   🗑️  Linhas duplicadas removidas: 0
   📊 Linhas finais: 66243
